## ネットワークの定義

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # affine
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
            
        return num_features

`num_flat_features()`　batch以外のものを一次元化するため

## ネットワークについての確認

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 5, 5])


## データセットの入力

In [5]:
input = torch.randn(1, 1, 32, 32, requires_grad=True)
out = net(input)
print(out)

tensor([[-0.0521, -0.2062,  0.0252, -0.0217,  0.0848,  0.0285, -0.0662, -0.0544,
         -0.0325,  0.0325]], grad_fn=<AddmmBackward>)


In [6]:
net.zero_grad()
out.backward(torch.randn(1, 10))
print(input.grad)

tensor([[[[ 9.6899e-06, -1.1500e-04, -4.1729e-04,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 1.1006e-04,  4.6535e-04,  5.9778e-04,  ...,  6.7381e-04,
            8.7619e-04,  8.5329e-04],
          [ 6.3392e-04,  3.9817e-04,  9.1723e-04,  ...,  5.6045e-04,
            1.0187e-03,  6.0665e-05],
          ...,
          [ 9.3652e-05, -3.6999e-04,  6.6226e-05,  ...,  1.8703e-04,
            5.3291e-04, -2.9264e-04],
          [ 9.1154e-06,  9.1452e-05, -6.3976e-05,  ..., -2.0227e-05,
            4.1780e-05, -7.8188e-05],
          [ 1.4517e-04, -1.0815e-04, -1.9949e-04,  ...,  2.4356e-04,
           -5.9754e-05,  3.2236e-05]]]])


- 勾配をゼロに初期化`net.zero_grad()`
- 出力を入力で微分
    - `out.backward()`(`torch.randn(1, 10)`はベクトル対策のやつ)
    - `input.grad`

## 損失の計算

In [7]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7648, grad_fn=<MseLossBackward>)


input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

In [8]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

計算過程の確認（`grad_fn`)  
`.next_functions`は覚えておきたい

## 逆伝搬

In [9]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0022, -0.0055,  0.0029, -0.0048, -0.0052, -0.0306])


biasにしたことに特に意味はなさそう

## 重みの更新

In [10]:
import torch.optim as optim

# create optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()